# Mathematical programming with Jupyter Notebooks

# Solving mixed integer programs with PuLP


*Copyright 2016, Pedro Belin Castellucci & Franklina Maria Bragion de Toledo, All rights reserved.*

*This Notebook is licenced under GLPv3 (https://opensource.org/licenses/GPL-3.0).*

In this notebook, we will learn how to create and solve mixed integer programs with PuLP in Python.
We will do this using examples. The first one is as follows:

Let $I$ be a set of items, $W = \{2, 3, 1, 5, 3\}$ be the multi-set (which allows repetition) of weigths of each item  in $i$ in $I$ and $C=10$ the weight capacity of a recipient. Let us write a model to maximise the number of items to put into the recipient without exceeding the weight capacity.

First, we need to define the binary variable $x_i$ which is equal to 1 if item $i$ should be included in the recipient. Then, we can write the model:

Max $x_1 + x_2 + x_3 + x_4 + x_5$ 

s. t.

$2x_1 + 3x_2 + 1x_3 + 5x_4 + 3x_5 \leq 10$

$x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}$

We will use PuLP to solve this model. The first step is to import the library and create a model object.

In [1]:
import pulp  # Library importing.

model = pulp.LpProblem('Our example', pulp.LpMaximize)  # Creating the model object.
print model

Our example:
MAXIMIZE
None
VARIABLES



We can see we have created a maximisation model, but it has no variables nor constraints in it so far. We need to create them and add the constraint into the model.

In [2]:
# Creating the binary variables:
x1 = pulp.LpVariable('x1', cat='Binary')
x2 = pulp.LpVariable('x2', cat='Binary')
x3 = pulp.LpVariable('x3', cat='Binary')
x4 = pulp.LpVariable('x4', cat='Binary')
x5 = pulp.LpVariable('x5', cat='Binary')

# Adding the constraint into the model:
model += 2*x1 + 3*x2 + 1*x3 + 5*x4 + 3*x5 <= 10

print model

Our example:
MAXIMIZE
None
SUBJECT TO
_C1: 2 x1 + 3 x2 + x3 + 5 x4 + 3 x5 <= 10

VARIABLES
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer



Note that we do not have to add the variables into the model explicitly, PuLP does that automatically, when a constraint (or the objective function) is added to the model. 

Now, our model has variables and a constraint but it is still missing the objective function.

In [3]:
# Adding the objective function:
model += x1 + x2 + x3 + x4 + x5

print model

Our example:
MAXIMIZE
1*x1 + 1*x2 + 1*x3 + 1*x4 + 1*x5 + 0
SUBJECT TO
_C1: 2 x1 + 3 x2 + x3 + 5 x4 + 3 x5 <= 10

VARIABLES
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer



The objective function and constraints are added into the model in a similar fashion. The key difference is the presence (or absence) of a relational operator ('<=' or '>=') for adding constraints (or the objective function). 

Now we have the model, let us ask PuLP to try to solve it and learn how to check the results.

In [12]:
# Asking PuLP to solve the model:
status = model.solve()

# Checking the status of the solution:
if status == pulp.LpStatusInfeasible:
    print 'The model is infeasible!'
elif status == pulp.LpStatusUnbounded:
    print 'The model is unbounded!'
elif status == pulp.LpStatusOptimal:
    print 'Optimal solution found! \o/'
else:
    print 'Ops! I do not know what happened!'

Optimal solution found! \o/
1


It is always a good idea to check the status of the solution to ensure meaning to the following operations. In our case, PuLP was able to find an optimal solution and we want to know what the solution is. Let us query for the value of the variable and the objective function.

In [ ]:
print 'Value of the variables:'
for v in model.variables():
    print '%s = %d' % (v, v.value())

print 'Number of items is %d.' % model.objective.value()

As an exercise, you can modify the model in order to get an infeasible solution and an unbounded solution. 

## Dealing with bigger models

So far we have learnt how to create a simple model, trying to solve it and query for results using PuLP. However, in real applications, there is a great ammount of data and models are much bigger than the one we used as an example. Imagine yourself, in our knapsack example, having to write the same constraint for 100 or 1000 items. We need tools to help us write models of some pratical importance. 

Let us start by creating a bigger example. Now, let $C=100$, the objective is to choose a subset of 100 items with the weigths defined as follows:

In [ ]:
from random import randrange
weights = [randrange(1, 20) for _ in range (100)]

print weights

In order to avoid typing all the weights manually, we will use the *dicts* function of PuLP. Right now, we have a list of weights and we will create another list, *items*, with the indexes of the $x$ variables to our model:

In [ ]:
# We need to explicity enumerate the items to use 'dicts' method:
items = range(100)

Then, we can use the *dicts* function to create all the variables at once:

In [ ]:
# Using the 'dicts' method to create the variables:
x = pulp.LpVariable.dicts('x', indexs=items, cat='Binary')

The objective function of our model consists of summing over all items. So, that is what we write:

In [ ]:
objectiveExpression = sum(x[i] for i in items)

And we can do the same for our constraint:

In [ ]:
constraintExpression = sum(weights[i]*x[i] for i in items) <= 100

Then, we just create the model object, add the expressions we have just created and ask PuLP to try to solve it:

In [ ]:
# Creating the model object:
model = pulp.LpProblem('Bigger example', pulp.LpMaximize)

# Adding the expressions:
model += objectiveExpression
model += constraintExpression

# Trying to solve it
status = model.solve()

# Checking the status of the solution:
if status == pulp.LpStatusInfeasible:
    print 'The model is infeasible!'
elif status == pulp.LpStatusUnbounded:
    print 'The model is unbounded!'
elif status == pulp.LpStatusOptimal:
    print 'Optimal solution found! \o/'
    
    print 'Value of the variables:'
    for v in model.variables():
        val = v.value()
        if val >= 0.99:
            print '%s = %d' % (v, v.value())

    print 'Number of items is %d.' % model.objective.value()
else:
    print 'Ops! I do not know what happened!'

Now that we have seen how to create and solve a model, you can try to use PuLP for solving the models discussed in class. Keep in mind that we just covered the basics, for the full documentation on PuLP you can refer to: https://pythonhosted.org/PuLP/. 
